# Local BYOC Churn

In [60]:
import boto3
import sagemaker
from sagemaker import get_execution_role

ecr_namespace = 'sagemaker-training-containers/'
prefix = 'pca'

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)

057716757052
us-east-2
arn:aws:iam::057716757052:role/service-role/AmazonSageMaker-ExecutionRole-20191128T110038
sagemaker-us-east-2-057716757052


Let's take a look at the Dockerfile which defines the statements for building our custom SageMaker training container:

## Docker 생성 및 ECR 푸시

In [61]:
! cp pca_byoc_train.py docker/code/

In [62]:
%%writefile docker/Dockerfile

FROM 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
    
# install python package
RUN pip install joblib


ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

ENV PATH="/opt/ml/code:${PATH}"

# Copy training code
COPY code/* /opt/ml/code/
 
WORKDIR /opt/ml/code

# ENTRYPOINT ["python", "pca_train.py"]
# In order to use SageMaker Env varaibles, use the statement below
ENV SAGEMAKER_PROGRAM pca_byoc_train.py

Overwriting docker/Dockerfile


In [63]:
import os
os.environ['account_id'] = account_id
os.environ['region'] = region
os.environ['ecr_repository_name'] = ecr_repository_name

In [64]:
%%sh

ACCOUNT_ID=${account_id}
REGION=${region}
REPO_NAME=${ecr_repository_name}

echo $REGION
echo $ACCOUNT_ID
echo $REPO_NAME


# Get the login command from ECR in order to pull down the Tensorflow-gpu:1.5 image
$(aws ecr get-login --registry-ids 257758044811 --region ${region} --no-include-email)



docker build -f docker/Dockerfile -t $REPO_NAME docker

docker tag $REPO_NAME $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest

$(aws ecr get-login --no-include-email --registry-ids $ACCOUNT_ID)

aws ecr describe-repositories --repository-names $REPO_NAME || aws ecr create-repository --repository-name $REPO_NAME

docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest



us-east-2
057716757052
sagemaker-training-containers/pca
Login Succeeded
Sending build context to Docker daemon  11.26kB
Step 1/8 : FROM 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
 ---> 30adb1aa9af5
Step 2/8 : RUN pip install joblib
 ---> Using cache
 ---> 0786847c4f79
Step 3/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 7d94abd2b857
Step 4/8 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 8696b5e742b3
Step 5/8 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> daba2554dce8
Step 6/8 : COPY code/* /opt/ml/code/
 ---> 9685910a18a5
Step 7/8 : WORKDIR /opt/ml/code
 ---> Running in c57a3ce60d1c
Removing intermediate container c57a3ce60d1c
 ---> ae3f15597ed8
Step 8/8 : ENV SAGEMAKER_PROGRAM pca_byoc_train.py
 ---> Running in 6350a8798bbd
Removing intermediate container 6350a8798bbd
 ---> 2838d3d55148
Successfully built 2838d3d55148
Successfully tagged sagemaker-training-containers/pca:latest
Login Succeeded
{
    "repositories

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [65]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

057716757052.dkr.ecr.us-east-2.amazonaws.com/sagemaker-training-containers/pca:latest


##  Churn 데이타 준비

In [66]:
%store -r

In [67]:
import pandas as pd

preprocessed_train_path_file = '{}/train.csv.out'.format(preprocessed_train_path)
pre_df = pd.read_csv(preprocessed_train_path_file, header=None)
pre_df.shape

(2333, 70)

In [68]:
import pandas as pd
# preprocessed_train_path_file = 's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-12-07-07-2020-08-12-07-07-08-229/train.csv.out'

churn_df = pd.read_csv(preprocessed_train_path_file, header=None)
churn_df.head()
train_y = churn_df.iloc[:,0]
train_X = churn_df.iloc[:,1:]

print("Shape of train_X: ", train_X.shape)
print("Shape of train_y: ", train_y.shape)

Shape of train_X:  (2333, 69)
Shape of train_y:  (2333,)


In [69]:
from sklearn import datasets
import os
import numpy as np


os.makedirs('./data', exist_ok =True)
np.savetxt('./data/churn-preprocessed.csv', train_X, delimiter=',',
           fmt='%1.5f'
          )

WORK_DIRECTORY = 'data'
prefix = 'Scikit-pca-custom'
train_input = sagemaker_session.upload_data(WORK_DIRECTORY,
                                            key_prefix="{}/{}".format(prefix, WORK_DIRECTORY)
                                           )
print("train_input: ", train_input)


train_input:  s3://sagemaker-us-east-2-057716757052/Scikit-pca-custom/data


# PCA 학습

In [70]:
%%time

import sagemaker

instance_type = 'local'
# instance_type = 'ml.m4.xlarge'

pca_estimator = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type= instance_type,
                                    base_job_name=prefix)

pca_estimator.set_hyperparameters(n_components= 2)

train_config = sagemaker.session.s3_input(train_input, content_type='text/csv')

pca_estimator.fit({'train': train_config})

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Creating tmpckke_plc_algo-1-uzg9p_1 ... 
Attaching to tmpckke_plc_algo-1-uzg9p_12mdone
algo-1-uzg9p_1  | 2020-08-14 06:40:48,080 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-uzg9p_1  | 2020-08-14 06:40:48,082 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-uzg9p_1  | 2020-08-14 06:40:48,091 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-uzg9p_1  | 2020-08-14 06:40:48,091 sagemaker-containers INFO     Module pca_byoc_train does not provide a setup.py. 
algo-1-uzg9p_1  | Generating setup.py
algo-1-uzg9p_1  | 2020-08-14 06:40:48,092 sagemaker-containers INFO     Generating setup.cfg
algo-1-uzg9p_1  | 2020-08-14 06:40:48,092 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-uzg9p_1  | 2020-08-14 06:40:48,092 sagemaker-containers INFO     Installing module with the following command:
algo-1-uzg9p_1  | /miniconda3/bin/python -m pip install . 
algo-1-uzg9p_1  | Pr

# PCA 트랜스포머

In [71]:
%store -r

In [72]:
preprocessed_train_path_file = '{}/train.csv.out'.format(preprocessed_train_path)

In [73]:

preproc_df = pd.read_csv(preprocessed_train_path_file, header=None)

preproc_df.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.0,0.119414,-0.596238,1.744368,0.978957,-0.028993,-0.893185,-0.801703,-1.982529,-1.530559,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,-1.852652,-0.596238,0.140284,-0.310405,0.970689,-0.689888,0.146389,1.232901,0.124852,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,1.181295,-0.596238,1.835130,0.185503,0.030988,-0.639063,1.568529,-0.063643,-0.846802,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.776769,-0.596238,0.216227,0.334276,0.136954,1.393914,1.394712,-0.634123,0.844596,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,-0.234547,1.508734,-0.459859,0.483049,-0.230929,0.224952,1.056954,0.921730,-0.810815,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [74]:
num_cols = preproc_df.shape[1]
num_cols

70

In [75]:


instance_type = 'local'
# instance_type = 'ml.m4.2xlarge'
transform_train_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-pca-train-output')

pca_model = pca_estimator.create_model(
    env={'TRANSFORM_MODE': 'feature-transform', 'LENGTH_COLS': str(num_cols)})

# scikit_learn_inferencee_model 에서 Train Transformer 생성
transformer_train = pca_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_train_output_path,
    accept = 'text/csv')


# Preprocess training input
transformer_train.transform(preprocessed_train_path_file, 
                            content_type='text/csv',                            
                           )

print('Waiting for transform job: ' + transformer_train.latest_transform_job.job_name)
transformer_train.wait()

preprocessed_train_path = transformer_train.output_path + transformer_train.latest_transform_job.job_name
print(preprocessed_train_path)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


Attaching to tmp1vpg5w23_algo-1-e3o0v_1
algo-1-e3o0v_1  | Processing /opt/ml/code
algo-1-e3o0v_1  | Building wheels for collected packages: pca-byoc-train
algo-1-e3o0v_1  |   Building wheel for pca-byoc-train (setup.py) ... done
algo-1-e3o0v_1  |   Created wheel for pca-byoc-train: filename=pca_byoc_train-1.0.0-py2.py3-none-any.whl size=9478 sha256=266dd92c50859625a4390b43d90185f5189f31c516b3430b9f7d07ae3e3a925a
algo-1-e3o0v_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-vh8dxswj/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-e3o0v_1  | Successfully built pca-byoc-train
algo-1-e3o0v_1  | Installing collected packages: pca-byoc-train
algo-1-e3o0v_1  | Successfully installed pca-byoc-train-1.0.0
algo-1-e3o0v_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-e3o0v_

# RealTime Endpoint

In [76]:
pca_infer_model = pca_estimator.create_model(
    env={'TRANSFORM_MODE': 'inverse-label-transform', 'LENGTH_COLS': '69'})


Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [77]:
# %%time 

# from time import gmtime, strftime
# timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# instance_type='local'
# # instance_type='ml.m4.2xlarge'


# endpoint_name= 'churn-model-pipeline-endpoint-' + timestamp_prefix

# deployed_model = pca_infer_model.deploy(
#     initial_instance_count=1, 
#     instance_type= instance_type, 
#     endpoint_name = endpoint_name,        
#     wait = True
# )

In [78]:
from sagemaker.predictor import csv_serializer

instance_type = 'local'
# instance_type = 'ml.m4.xlarge'

est_predictor = pca_estimator.deploy(
    initial_instance_count = 1,
    instance_type = instance_type,
    model_name = pca_infer_model.name,
    serializer=csv_serializer
)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


Attaching to tmpakjwyiug_algo-1-oum60_1
algo-1-oum60_1  | Processing /opt/ml/code
algo-1-oum60_1  | Building wheels for collected packages: pca-byoc-train
algo-1-oum60_1  |   Building wheel for pca-byoc-train (setup.py) ... done
algo-1-oum60_1  |   Created wheel for pca-byoc-train: filename=pca_byoc_train-1.0.0-py2.py3-none-any.whl size=9476 sha256=f888642ca3c0a776299f38bfa1de044dadeb28e9ed098babe1d5a793fce1d6e8
algo-1-oum60_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-n8x0qe76/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-oum60_1  | Successfully built pca-byoc-train
algo-1-oum60_1  | Installing collected packages: pca-byoc-train
algo-1-oum60_1  | Successfully installed pca-byoc-train-1.0.0
algo-1-oum60_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-oum60_

In [79]:
# sample = train_X[0:1].reshape(1,-1) # Single Sample (1,-1)
sample = train_X[0:2]
print("Shape of sample: ", sample.shape)
sample.values
result = est_predictor.predict(sample.values)
result

Shape of sample:  (2, 69)
algo-1-oum60_1  | 2020-08-14 06:41:02,853 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-oum60_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-oum60_1  |   import imp
algo-1-oum60_1  | input_fn: Starting
algo-1-oum60_1  | type of input_data:  <class 'str'>
algo-1-oum60_1  | request_content_type:  text/csv
algo-1-oum60_1  | 2020-08-14 06:41:03,228 INFO - root - input_fn: 
algo-1-oum60_1  | 2020-08-14 06:41:03,228 INFO - root - shape of requested data: '(2, 69)'
algo-1-oum60_1  | 2020-08-14 06:41:03,285 INFO - root - requested data: '         0   ...   68
algo-1-oum60_1  | 0  0.119414  ...  0.0
algo-1-oum60_1  | 1 -1.852652  ...  0.0
algo-1-oum60_1  | 
algo-1-oum60_1  | [2 rows x 69 columns]'
algo-1-oum60_1  | prdict_fn: Staring
algo-

b'-0.8230854363168091,-0.10881113184516987\n-0.34347425332246134,0.0914233018366202\n'

algo-1-oum60_1  | 172.18.0.1 - - [14/Aug/2020:06:41:03 +0000] "POST /invocations HTTP/1.1" 200 81 "-" "-"


In [80]:
result

b'-0.8230854363168091,-0.10881113184516987\n-0.34347425332246134,0.0914233018366202\n'

algo-1-oum60_1  | 2020-08-14 06:42:31,604 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-oum60_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-oum60_1  |   import imp
algo-1-oum60_1  | 172.18.0.1 - - [14/Aug/2020:06:42:31 +0000] "GET /ping HTTP/1.1" 200 0 "-" "-"
algo-1-oum60_1  | 172.18.0.1 - - [14/Aug/2020:06:42:31 +0000] "GET /execution-parameters HTTP/1.1" 404 232 "-" "-"
algo-1-oum60_1  | input_fn: Starting
algo-1-oum60_1  | type of input_data:  <class 'str'>
algo-1-oum60_1  | request_content_type:  text/csv
algo-1-oum60_1  | 2020-08-14 06:42:32,131 INFO - root - input_fn: 
algo-1-oum60_1  | 2020-08-14 06:42:32,131 INFO - root - shape of requested data: '(2333, 21)'
algo-1-oum60_1  | 2020-08-14 06:42:32,159 INFO - root - requested data: '      0   ...  